In [9]:
import csv

# Leer puntos críticos
puntos_criticos = []
with open('crucial_points.csv', 'r', encoding='utf-8') as archivo_csv:
    lector = csv.DictReader(archivo_csv)
    for fila in lector:
        punto = {
            'nombre': fila['Sector'],
            'latitud': float(fila['Norte (Inicial)']),
            'longitud': float(fila['Este (Inicial)'])
        }
        puntos_criticos.append(punto)

# Leer embalses
embalses = []
with open('embalses_con_coordenadas.csv', 'r', encoding='utf-8') as archivo_csv:
    lector = csv.DictReader(archivo_csv)
    for fila in lector:
        # Verificar si 'Latitud' y 'Longitud' no están vacíos
        if fila['Latitud'] and fila['Longitud']:
            try:
                # Reemplazar comas por puntos si es necesario
                latitud = float(fila['Latitud'].replace(',', '.'))
                longitud = float(fila['Longitud'].replace(',', '.'))
                embalse = {
                    'nombre': fila['Nombre de la Presa'],
                    'latitud': latitud,
                    'longitud': longitud
                }
                embalses.append(embalse)
            except ValueError:
                print(f"Advertencia: Datos inválidos en la fila para el embalse '{fila['Nombre de la Presa']}'. Se omitirá esta entrada.")
        else:
            print(f"Advertencia: Latitud o Longitud faltante para el embalse '{fila['Nombre de la Presa']}'. Se omitirá esta entrada.")


In [10]:
import math

def distancia_haversine(lat1, lon1, lat2, lon2):
    # Radio de la Tierra en kilómetros
    R = 6371.0
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    a = math.sin(delta_phi / 2.0)**2 + \
        math.cos(phi1) * math.cos(phi2) * \
        math.sin(delta_lambda / 2.0)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c  # Distancia en kilómetros


In [13]:
import heapq

# Construir el grafo
grafo = {}

for punto in puntos_criticos:
    grafo[punto['nombre']] = {}
    for embalse in embalses:
        distancia = distancia_haversine(
            punto['latitud'], punto['longitud'],
            embalse['latitud'], embalse['longitud']
        )
        grafo[punto['nombre']][embalse['nombre']] = distancia

# Implementar Dijkstra
def dijkstra(grafo, inicio):
    distancias = {nodo: float('inf') for nodo in grafo}
    distancias[inicio] = 0
    cola_prioridad = [(0, inicio)]
    while cola_prioridad:
        distancia_actual, nodo_actual = heapq.heappop(cola_prioridad)
        if distancia_actual > distancias[nodo_actual]:
            continue
        for vecino, peso in grafo[nodo_actual].items():
            distancia = distancia_actual + peso
            if distancia < distancias.get(vecino, float('inf')):
                distancias[vecino] = distancia
                heapq.heappush(cola_prioridad, (distancia, vecino))
    return distancias


In [21]:
import tkinter as tk
from tkinter import ttk

class AplicacionEmbalses:
    def __init__(self, root):
        self.root = root
        self.root.title("Embalses más Cercanos")

        # Combobox para seleccionar el punto crítico
        self.var_punto = tk.StringVar()
        nombres_puntos = [p['nombre'] for p in puntos_criticos]
        self.combo_puntos = ttk.Combobox(root, textvariable=self.var_punto, values=nombres_puntos)
        self.combo_puntos.grid(row=0, column=0, padx=10, pady=10)

        # Botón para encontrar el embalse más cercano
        self.boton_buscar = tk.Button(root, text="Encontrar Embalse Más Cercano", command=self.encontrar_embalse)
        self.boton_buscar.grid(row=0, column=1, padx=10, pady=10)

        # Etiqueta para mostrar el resultado
        self.etiqueta_resultado = tk.Label(root, text="")
        self.etiqueta_resultado.grid(row=1, column=0, columnspan=2, padx=10, pady=10)

    def encontrar_embalse(self):
        nombre_punto = self.var_punto.get()
        if not nombre_punto:
            self.etiqueta_resultado.config(text="Por favor, selecciona un punto crítico.")
            return

        # Obtener el punto crítico seleccionado
        punto = next((p for p in puntos_criticos if p['nombre'] == nombre_punto), None)
        if punto is None:
            self.etiqueta_resultado.config(text="Punto crítico no encontrado.")
            return

        # Construir un grafo temporal para el punto seleccionado
        grafo_temporal = {punto['nombre']: {}}
        for embalse in embalses:
            distancia = distancia_haversine(
                punto['latitud'], punto['longitud'],
                embalse['latitud'], embalse['longitud']
            )
            grafo_temporal[punto['nombre']][embalse['nombre']] = distancia

            # Agregar el embalse como nodo sin vecinos
            if embalse['nombre'] not in grafo_temporal:
                grafo_temporal[embalse['nombre']] = {}

        # Ejecutar Dijkstra
        distancias = dijkstra(grafo_temporal, punto['nombre'])

        # Filtrar las distancias solo a los embalses
        distancias_embalses = {e['nombre']: distancias[e['nombre']] for e in embalses if e['nombre'] in distancias}
        if not distancias_embalses:
            self.etiqueta_resultado.config(text="No se encontraron embalses accesibles.")
            return

        embalse_cercano = min(distancias_embalses, key=distancias_embalses.get)

        texto_resultado = f"El embalse más cercano a {punto['nombre']} es {embalse_cercano} a {distancias_embalses[embalse_cercano]:.2f} km."
        self.etiqueta_resultado.config(text=texto_resultado)

        # Obtener los datos del embalse más cercano
        embalse_seleccionado = next((e for e in embalses if e['nombre'] == embalse_cercano), None)
        if embalse_seleccionado:
            # Llamar a crear_mapa para generar el mapa
            crear_mapa(punto, embalse_seleccionado)
            self.etiqueta_resultado.config(text=texto_resultado + "\nEl mapa ha sido guardado como 'ruta.html'.")
        else:
            self.etiqueta_resultado.config(text=texto_resultado + "\nNo se pudo generar el mapa.")


# Ejecutar la aplicación
root = tk.Tk()
app = AplicacionEmbalses(root)
root.mainloop()
